#### 实验介绍：
##### 将介绍如何通过 Amazon Bedrock 和 CrewAI 框架利用 Anthropic 的 Claude-3 语言模型的强大功能，从而将大型语言模型和代理工作流的定义无缝集成到自定义 AI 应用程序中

In [1]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

In [2]:
chat = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={"temperature": 0.1},
    region_name="us-east-1"
)

In [3]:
messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
chat.invoke(messages)

AIMessage(content="Voici la traduction en français :\n\nJ'aime la programmation.", additional_kwargs={'usage': {'prompt_tokens': 20, 'completion_tokens': 21, 'total_tokens': 41}}, response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 20, 'completion_tokens': 21, 'total_tokens': 41}}, id='run-062cd5ae-2fb2-4cb4-83a3-6c30d39de27f-0')

##### 流式输出

In [4]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

Voici la traduction en français :

J'aime la programmation.

##### 使用ChatPromptTemplate

In [5]:
from langchain_core.prompts import ChatPromptTemplate

In [6]:
system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Chinese",
        "text": "I love Python",
    }
)

AIMessage(content='我喜欢Python', additional_kwargs={'usage': {'prompt_tokens': 22, 'completion_tokens': 10, 'total_tokens': 32}}, response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 22, 'completion_tokens': 10, 'total_tokens': 32}}, id='run-ee7579ff-7e9d-4cb6-ada9-4e19939e4c45-0')

##### ChatAnthropic 也支持async和streaming功能:

In [7]:
prompt = ChatPromptTemplate.from_messages([("human", "Tell me a joke about {topic}")])
chain = prompt | chat
await chain.ainvoke({"topic": "bear"})

AIMessage(content="Here's a bear joke for you:\n\nWhy did the bear dissolve in water?\nBecause it was a polar bear!", additional_kwargs={'usage': {'prompt_tokens': 13, 'completion_tokens': 29, 'total_tokens': 42}}, response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 13, 'completion_tokens': 29, 'total_tokens': 42}}, id='run-e502f851-651e-4e91-8bdf-a3c0e060b324-0')

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [("human", "Give me a list of famous tourist attractions in Japan")]
)
chain = prompt | chat
for chunk in chain.stream({}):
    print(chunk.content, end="", flush=True)

Here are some of the most famous tourist attractions in Japan:

1. Mount Fuji - The iconic snow-capped mountain and highest peak in Japan.
2. Imperial Palace and Gardens (Tokyo) - The primary residence of the Emperor of Japan.
3. Sensoji Temple (Tokyo) - An ancient Buddhist temple and one of Tokyo's most colorful and popular temples.
4. Fushimi Inari Shrine (Kyoto) - Known for its thousands of vermilion torii gates.
5. Hiroshima Peace Memorial Park and Museum - A memorial to the atomic bombing of Hiroshima in 1945.
6. Kinkakuji (Golden Pavilion) (Kyoto) - A Zen Buddhist temple with a golden pavilion set in a pond.
7. Nijo Castle (Kyoto) - A famous wooden castle known for its "nightingale floors" that squeak to warn of intruders.
8. Himeji Castle - One of Japan's most impressive and well-preserved feudal castles.
9. Jigoku-Meguri (Hell Tour) (Beppu) - A tour of hot springs and steaming pools resembling a hellish landscape.
10. Tsukiji Fish Market (Tokyo) - The world's largest fish marke

#### 接下来将使用CrewAI建立了一个名为“ 资深行业分析师“  的代理，旨在发现人工智能和数据科学的前沿发展。然后，它分配了一项任务，对 2024 年最新的人工智能进展进行全面分析，预期输出是一份要点报告。最后，它使用代理和任务创建一个“Crew”实例，并执行工作流

In [9]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

researcher = Agent(
 role='资深行业分析师',
 goal='揭示人工智能和数据科学的前沿发展',
 backstory="你在一家领先的科技智库工作…",
 verbose=True,
 allow_delegation=False,
 llm=chat,
 tool=[SerperDevTool()]
)
task1 = Task(
 description="对2024年人工智能的最新进展进行全面分析…",
 expected_output="中文分析报告",
 agent=researcher
)
crew = Crew(
 agents=[researcher],
 tasks=[task1],
 verbose=2,
)

In [10]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: 资深行业分析师
 [INFO]: == Starting Task: 对2024年人工智能的最新进展进行全面分析…


> Entering new CrewAgentExecutor chain...
好的,我会尽力提供一份全面的2024年人工智能最新进展分析报告。

Final Answer:

2024年人工智能最新进展分析报告

引言

人工智能技术在过去几年取得了长足进步,并在各行各业得到广泛应用。2024年,人工智能将继续快速发展,在多个领域出现突破性进展。本报告将全面分析2024年人工智能最新进展,包括计算能力、算法模型、应用场景等多个方面。

一、计算能力的飞速提升

计算能力是推动人工智能发展的关键驱动力。2024年,全球超级计算机性能将再次大幅提升,量子计算机也将开始投入使用。这将为训练大规模人工智能模型提供强大算力支持。同时,专用AI芯片的性能也将持续提高,使得人工智能系统在边缘设备上的运行更加高效。

二、通用人工智能模型取得突破

2024年,通用人工智能模型将取得重大突破。基于大规模语料训练的大型语言模型将进一步优化,在自然语言理解、生成、推理等方面表现出超人类水平。同时,结合多模态数据(图像、视频等)训练的多模态模型也将问世,显著提升人工智能系统的认知和理解能力。

三、人工智能在重要领域的创新应用

2024年,人工智能将在医疗健康、智能制造、智能交通等重要领域出现大量创新应用。

1) 医疗健康领域,人工智能将广泛应用于疾病诊断、药物研发、医疗影像分析等环节,大幅提高医疗质量和效率。

2) 智能制造领域,人工智能将深度融入产品设计、工艺优化、质量控制等全流程,推动制造业向智能化、柔性化转型。

3) 智能交通领域,自动驾驶技术将取得重大进展,无人驾驶汽车将开始在特定场景下投入使用。同时,人工智能也将优化交通规划和调度,显著提升交通运行效率。

四、人工智能安全与伦理问题日益受重视

随着人工智能技术的不断发展和广泛应用,人工智能系统的安全性和伦理问题也将日益受到重视。2024年,人工智能系统的可解释性、公平性、隐私保护等问题将成为研究热点。同时,人工智能治理框架也将逐步完善,以确保人工智能技术的可控发展和负责任使用。

总结

2024年将是人工智能发展的重要一年。计算能力的